## CrewAI Demo
In this notebook, we will take a look at how we can use crewAI to create crew of autonomous agents by creating a crew to get latest currency information from internet and then apply mathematical operations on it.

Let's first start by installing the required dependencies

In [ ]:
%pip install crewai requests 'crewai[tools]'

Let’s import the methods and modules which we are going to use

In [ ]:
# Import statements
from crewai import Crew, Agent, Task
from textwrap import dedent
import os
import json
import requests

Don’t forget to store your openai API key in environment variables with name `OPENAI_API_KEY` otherwise it won’t work!

In [ ]:
os.environ["OPENAI_API_KEY"] = openai_secret;

Now let’s create our custom tools, one for mathematical calculation and one for internet search

Here we will use `@tool` decorator from Langchain to specify a tool name and description so that our agents can use it. These tools are just simple functions where we will write our business logic

In [ ]:
from langchain.tools import tool
class Tools:
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"

    @tool("Search the internet")
    def search_internet(query):
        """Useful to search the internet 
        about a a given topic and return relevant results"""
        top_result_to_return = 2
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': serper_secret,
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()['organic']
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append('\n'.join([
                    f"Title: {result['title']}", f"Link: {result['link']}",
                    f"Snippet: {result['snippet']}", "\n-----------------"
                ]))
            except KeyError:
                next

        return '\n'.join(string)

Now let’s create our agents aka AI employees

In [ ]:
class Agents:
  def mathsAgent():
    return Agent(
    role='Maths expert',
    goal='Perform the calculations on the numbers given to you',
    backstory='An maths expert working with numbers who have experience solving complex and basic maths problems',
    tools=[Tools.calculate],
    verbose=True,
    allow_delegation=False # if it's true then agent will be able to delegate task to another agent
  )
  def searchAgent():
    return Agent(
    role='Financial Expert',
    goal='Search internet and give the current value of given currency in INR',
    backstory='A stock market expert who have very good knowledge about every currency and have given accurate currency values',
    tools=[Tools.search_internet],
    verbose=True,
    allow_delegation=False
  )

And now let’s create the tasks which we will give to these agents

In [ ]:
class Tasks:
  def mathsTask(agent,operation):
    return Task(
      description=dedent(f"""
          You are given a currency value and you have to perform given operation on that number
          Here is the operation:
          {operation}
        """),
      agent=agent,
      expected_output="Final answer after performing mathematical operation"
  )
  def searchTask(agent,stock):
    return Task(
      description=dedent(f"""
        you have given a currency name and you have to search internet and find the current value of this currency in INR
        Here is the currency name:
        {stock}
      """),
      agent=agent,
      expected_output="currency value in INR as a string"
    )

Let’s create our crew class, we will call it `MathsCrew` for now but you can give any name you want because it’s your company 😎

Remember that the sequence of tasks matter here because it is a sequential process.

In [ ]:
class MathsCrew:
  def __init__(self,stock,operation):
    self.stock = stock
    self.operation = operation
  def run(self):
		# Agents
    mathsAgent = Agents.mathsAgent()
    searchAgent = Agents.searchAgent()
		# Tasks
    searchTask = Tasks.searchTask(agent=searchAgent,stock=stock)
    mathsTask = Tasks.mathsTask(agent=mathsAgent,operation=operation)
		# Create crew
    crew = Crew(
      agents=[searchAgent,mathsAgent],
      tasks=[searchTask,mathsTask],
      verbose=2, # You can set it to 1 or 2 to different logging levels
    )
		# Run the crew
    result = crew.kickoff()
    print(result)

Lastly, add a code to take input from user and run the crew

In [ ]:
print("Welcome to currency agent!")
print("---------------------------")
stock = input("Enter currency name:")
operation = input("Enter operation:")

mathsCrew = MathsCrew(stock,operation)
result = mathsCrew.run()
print(result)